# Blue Brain BioExplorer
![](../../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer, MovieMaker
import os

resource_folder = os.path.abspath('../../../tests/test_files')
    
url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
mm = MovieMaker(be)
version = be.version()
print('Version ' + version)

sequence_name = 'low_glucose'

## Data loading

In [ ]:
'''Reset current scene'''
status = be.reset()

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=True,
    model_visibility_on_creation=False)

In [ ]:
cache_folder = os.path.join(resource_folder, 'caches', version)

In [ ]:
cache_filename = os.path.join(cache_folder, sequence_name + '_scenario.bioexplorer')
print('Loading ' + cache_filename)
dummy = core.add_model(cache_filename)

In [ ]:
nb_models = len(core.scene.models)
if nb_models != 152:
    print('ERROR: Not all models have been loaded!!! Expected 152, found ' + str(nb_models))
else:
    print('All good!')

In [ ]:
'''Show loaded models'''
status = be.set_models_visibility(True)

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

## Image settings

In [ ]:
image_size = [3840, 2160]
image_samples_per_pixel = 64

output_folder = os.path.join('/tmp', 'frames/', version, sequence_name)

command_line='mkdir -p ' + output_folder
os.system(command_line)
command_line='ls ' + output_folder

if os.system(command_line) != 0:
    print('ERROR: Failed to create output folder')

## Snapshots

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_DIFFUSE,
    specular_exponent=50.0, user_parameter=1.0)

In [ ]:
def set_all_materials_to_grey():
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        palette = list()
        for p in range(nb_materials):
            palette.append([0.25,0.25,0.25])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids, 
            diffuse_colors=palette, specular_colors=palette)

In [ ]:
def set_glycans_materials(high_mannose=False, complex=False, hybrid=False, o_glycans=False):
    glycans_colors = [[0, 1, 1], [1, 1, 0], [1, 0, 1], [0.2, 0.2, 0.7]]

    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        if be.NAME_GLYCAN_HIGH_MANNOSE in model_name and high_mannose:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[0])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )
        if be.NAME_GLYCAN_COMPLEX in model_name and complex:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[1])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )            
        if be.NAME_GLYCAN_HYBRID in model_name and hybrid:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[2])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )                
        if be.NAME_GLYCAN_O_GLYCAN in model_name and o_glycans:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[3])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )

### Frame 600 - Spike and glycans

In [ ]:
status = core.set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[205.7975332343745, 74.45357835395305, -108.21803523500476],
    target=[205.7975332343745, 74.45357835395305, -116.44005546175096]
)

In [ ]:
set_all_materials_to_grey()
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder, base_name='00600')

In [ ]:
set_glycans_materials(complex=True)
mm.create_snapshot(
    renderer='bio_explorer',
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder, base_name='00600a')

In [ ]:
set_glycans_materials(o_glycans=True)
mm.create_snapshot(
    renderer='bio_explorer',
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder, base_name='00600b')

In [ ]:
set_glycans_materials(high_mannose=True)
mm.create_snapshot(
    renderer='bio_explorer',
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder, base_name='00600c')

### Frame 2000 - Immune system

In [ ]:
status = mm.set_camera(
    direction=[-0.47143351766367525, -0.005080672314313481, -0.8818869684897763],
    origin=[238.16299438476562, 46.43607711791992, 372.5840759277344],
    up=[-0.0023952302678021103, 0.9999870933011259, -0.004480636464793384]    
)

In [ ]:
def apply_immune_colors():
    import seaborn as sns
    shading_mode = be.SHADING_MODE_BASIC
    specular_exponent = 50
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)

        if be.NAME_GLUCOSE in model_name:
            palette = sns.color_palette('Blues', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_LACTOFERRIN in model_name:
            palette = sns.color_palette('afmhot', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_DEFENSIN in model_name:
            palette = sns.color_palette('plasma_r', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_SURFACTANT_HEAD in model_name:
            palette = sns.color_palette('OrRd_r', nb_materials)
            emission = 0
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids,
                palette=palette, emission=emission)
        if be.NAME_COLLAGEN in model_name:
            material_ids = list(be.get_material_ids(model_id)['ids'])
            nb_materials = len(material_ids)
            palette = list()
            emissions = list()
            for i in range(nb_materials):
                palette.append([1,1,1])
                emissions.append(0.1)
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette,
                emissions=emissions
            )

In [ ]:
set_all_materials_to_grey()
apply_immune_colors()

In [ ]:
mm.create_snapshot(
    renderer='bio_explorer',
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder, base_name='02000')

### Frame 2600 - Surfactant head

In [ ]:
set_all_materials_to_grey()
apply_immune_colors()

In [ ]:
status = core.set_camera(
    orientation=[0.08576727578094712, 0.4753964796792586, -0.054913177571414236, 0.873857370766359],
    position=[-9.82699966430664, 110.72000122070312, 60.944000244140625],
    target=[-10.64799976348877, 110.92200469970703, 60.4109992980957]
)

In [ ]:
mm.create_snapshot(
    renderer='bio_explorer',
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder, base_name='02600')